In [16]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
import astropy.constants as const
import os, sys, time
import astropy.units as u
from fractions import Fraction

In [18]:
rc('text',usetex=True)
rc('font',**{'family':'serif','serif':['Times New Roman'],'size':14})#,'weight':'bold'})

In [19]:
current_path = os.getcwd()
splt_path = current_path.split("/")
top_path_idx = splt_path.index('DetectorDesignSensitivities')
top_directory = "/".join(splt_path[0:top_path_idx+1])
load_directory = top_directory + '/LoadFiles/InstrumentFiles/'

sys.path.insert(0,top_directory + '/Functions')
import StrainandNoise as SnN
import StrainandNoise_v2 as SnN_v2
import SNRcalc as SNc
import SNRcalc_v2 as SNc_v2

In [77]:
Nanograv = SnN_v2.PTA('NANOGrav')

In [79]:
Nanograv.Defaut_Setup()

In [81]:
print(Nanograv.name)
print(Nanograv.inst_var_dict)

NANOGrav
{'T_obs': {'val': <Quantity 1.57788e+08 s>, 'min': <Quantity 3.15576e+08 s>, 'max': <Quantity 9.46728e+08 s>}, 'N_p': {'val': 30, 'min': 18, 'max': 45}, 'rms': {'val': <Quantity 1.e-07 s>, 'min': <Quantity 1.e-08 s>, 'max': <Quantity 1.5e-07 s>}, 'cadence': {'val': <Quantity 8.26719577e-07 1 / s>, 'min': <Quantity 1.65343915e-06 1 / s>, 'max': <Quantity 4.13359788e-07 1 / s>}}


In [97]:
LISA = SnN_v2.SpaceBased('LISA')

In [98]:
LISA.Default_Setup()

In [99]:
print(LISA.inst_var_dict)

{'T_obs': {'val': <Quantity 1.262304e+08 s>, 'min': None, 'max': None}, 'L': {'val': <Quantity 2.5e+09 m>, 'min': None, 'max': None}, 'A_acc': {'val': <Quantity 3.e-15 m / s2>, 'min': None, 'max': None}, 'f_acc_break_low': {'val': <Quantity 0.0004 Hz>, 'min': None, 'max': None}, 'f_acc_break_high': {'val': <Quantity 0.008 Hz>, 'min': None, 'max': None}, 'A_IMS': {'val': <Quantity 1.e-11 m>, 'min': None, 'max': None}, 'f_IMS_break': {'val': <Quantity 0.002 Hz>, 'min': None, 'max': None}}


In [100]:
print(LISA.Get_Param_Dict('f_acc_break_high')['val'])

0.008 Hz
